### Yawn Detection

In [1]:
import cv2
import dlib
import numpy as np

PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
predictor_shape = dlib.shape_predictor(PREDICTOR_PATH)

detector_face = dlib.get_frontal_face_detector()


def get_landmarks(img):

    rects = detector_face(img,1)

    if len(rects) > 1:
        return "error"
    if len(rects) == 0:
        return "error"
    return np.matrix([[p.x,p.y] for p in predictor_shape(img,rects[0]).parts()])

def annonate_landmarks(img,landmarks):
    img = img.copy()

    for idx,point in enumerate(landmarks):
        pos = (point[0,0],point[0,1])
        cv2.putText(img,str(idx),pos,
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.4,
                    color=(0,0,255))
        cv2.circle(img,pos,3,color=(0,255,255))
    return img

def top_lip(landmarks):
    top_lip_pts = []
    for i in range(50,53):
        top_lip_pts.append(landmarks[i])
    for i in range(61,64):
        top_lip_pts.append(landmarks[i])
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    top_lip_mean = np.mean(top_lip_pts,axis=0)
    return int(top_lip_mean[:,1])

def bottom_lip(landmarks):
    bottom_lip_pts = []
    for i in range(65,68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56,59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts,axis=0)
    return int(bottom_lip_mean[:,1])

def mouth_open(image):
    landmarks = get_landmarks(image)

    if landmarks == "error":
        return  image,0
    # drawing new image with landmarks -> finding
    # distance between them -> returning
    imageWithLandmarks = annonate_landmarks(image,landmarks)
    top_lip_center = top_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lipDistance = abs(top_lip_center- bottom_lip_center)
    return  imageWithLandmarks,lipDistance


capture = cv2.VideoCapture(0)
no_of_yawn = 0
yawn_status = False

# checking whether mouth is open beyond a distance
# if its open then text is placed

while True:
    ret,frame = capture.read()
    imageLandmarks,lipDistance = mouth_open(frame)

    previous_yawn_status = yawn_status

    # if lip distance is greater than 25 text is printed
    if lipDistance > 25:
        yawn_status = True


        cv2.putText(frame, "Subject is yawning", (50,450),
                    cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        output_text = "Yawn Count: "+str(no_of_yawn+1)

        cv2.putText(frame,output_text,(50,50),cv2.FONT_HERSHEY_COMPLEX,1,
                     (0,255,127),2)
    else:
        yawn_status = False

    # yawn counts are incremented
    if previous_yawn_status == True and yawn_status == False:
        no_of_yawn = no_of_yawn+1

    cv2.imshow("Live Landmarks",imageLandmarks)
    cv2.imshow("Yawn detection",frame)

    if cv2.waitKey(1) == 13:
        break
capture.release()
cv2.destroyAllWindows()







/home/robot/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
